In [ ]:
#бот для ответов

import re
import urllib.request
import telebot
#pip install pytelegrambotapi
bot = telebot.TeleBot('1058517200:AAFulHMZuuRwJf8hX7yFtN9cyfBM1lYyNq0')

id_list=[]

@bot.message_handler(content_types=['text'])
def start_message(message):
    if message.text == '/start':
        bot.send_message(message.chat.id, 'Привет, '+message.from_user.first_name)
        bot.send_message(message.chat.id, 'Ты хочешь получать обновления тикетов в Телеграм?') #, reply_markup=keyboard1
        bot.register_next_step_handler(message, get_agree);
    else:
        bot.send_message(message.from_user.id, 'Напиши /start');

def get_agree(message): 
    if message.text.lower() in ('da','yes','да','хочу'):
        with open('chat_id.txt', 'a') as f: f.write("%s\n" % message.chat.id)
        bot.send_message(message.from_user.id, 'Вы подключены к рассылке');
    else:
        bot.send_message(message.from_user.id, 'Ну и следи сам за тикетами!');

bot.polling()

In [1]:
#получить chat_id  с pythonanywhere

def get_chat_id_list():
    from requests import Session
    from bs4 import BeautifulSoup as bs
    import logging
    import requests
    import re

    #logging.basicConfig(level=logging.DEBUG)  
    with Session() as s:
        s.headers['user-agent'] = 'myapp'
        site = s.get("https://www.pythonanywhere.com/login/")
        bs_content = bs(site.content, "html.parser")
        token = bs_content.find("input", {"name":"csrfmiddlewaretoken"})["value"]
        login_data = {"auth-username":"vzhyvov","auth-password":"Vova__99159915", "csrfmiddlewaretoken":token,"login_view-current_step":"auth"}
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36',
                'Referer': 'https://www.pythonanywhere.com/login/'}
        s.post("https://www.pythonanywhere.com/login/",login_data,headers=headers)
        parameters = s.get("https://www.pythonanywhere.com/user/VladZ/shares/1eb8edbbca074a9b9bedbc67c6a27fd5/")
        s=parameters.content.decode("utf-8")
        result = s[s.find('<code>')+len('<code>'):s.rfind('</code>')]
        result=result.splitlines()
        return list(set(result))

In [2]:
#бот собирает джира таски и отправляет
import schedule
import time
from jira import JIRA
import re
import urllib.request
import telebot

def run_jira():
    jira_options = {'server': 'https://jira.axiomsl.us/'}
    jira = JIRA(options=jira_options, basic_auth=("vzhyvov", "Vova___99159915"))

    jql = 'project = AXLATAM' 
    issues_list = jira.search_issues(jql)

    #AND Updated>2019-11-10
    issues = jira.search_issues('project=AXLATAM AND updated>-1d', maxResults=None,
                               fields=["summary,assignee,status,comment"])
    #AND updated >-1h - за послед час
    issue_link=[]
    summary=[]
    status=[]
    assignee=[]
    comment_list=[]
    for issue in issues:
        issue_link.append('https://jira.axiomsl.us/browse/'+issue.key)
        summary.append(issue.fields.summary)
        assignee.append(issue.fields.assignee.displayName)
        status.append(issue.fields.status.name)
        for comment in jira.comments(issue):
            com=comment.body
            comment_list.append(com)

    message=''
    for i in range(len(summary)):
        message=message+issue_link[i]+'%0A*'+summary[i]+'*%0A'+'Status: '+status[i]+'   '+'Assignee: '+assignee[i]+'%0A'+'**********************************'+'%0A'
    message=message.replace('\r\n','%0A')
    message=message.encode('ascii',errors='ignore').decode("utf-8")
    
#    updted_list=[]
#    with open('chat_id.txt', 'r') as f:
#        for i in f:
#            updted_list.append(i.rstrip())
#    updted_list=set(updted_list)
    if len(summary)==0:
        print('nothing')
    else: 
        for chatId in get_chat_id_list():
            urllib.request.urlopen("""
                https://api.telegram.org/bot{API_TOKEN}/sendMessage?chat_id={CHAT_ID}&text={TEXT}
            """.format(
                API_TOKEN = '1058517200:AAFulHMZuuRwJf8hX7yFtN9cyfBM1lYyNq0',
                CHAT_ID = chatId,
                TEXT = message
            ))
            time.sleep(3)




    #schedule.every(10).seconds.do(job)
    #schedule.every().hour.do(job)

schedule.every().minute.do(run_jira)
while 1:
    schedule.run_pending()
    time.sleep(1)

HTTPError: HTTP Error 403: Forbidden